In [ ]:
# Una empresa quiere lanzar un nuevo producto al mercado. Antes de hacerlo, desea predecir si este producto 
# será un éxito o un fracaso basándose en tres características: 
# 1. Costo de producción 
# 2. Precio de venta al público 
# 3. Gasto en publicidad 
# Tienes acceso a un conjunto de datos con 1000 registros que contienen estos parámetros y el resultado de 
# ventas de productos anteriores (Éxito/Fracaso). 

In [3]:
import numpy as np
import pandas as pd

In [ ]:
# Semilla para reproducibilidad
np.random.seed(42)

# Crear datos aleatorios para las características 
costo_produccion = np.random.uniform(50, 200, 1000)  # costos entre 50 y 200 ( 1000 productos )
precio_venta = costo_produccion + np.random.uniform(20, 80, 1000)  # precio de venta mayor al costo de producción 
gasto_publicidad = np.random.uniform(500, 5000, 1000)  # gasto en publicidad entre 500 y 5000 
 
# Suponemos que un producto es un éxito si el precio de venta es menos de 1.5 veces el costo y el gasto en 
#publicidad es mayor a 2500 
# Esto es solo para fines de ejemplo y no tiene una base real. 



In [6]:
etiquetas = np.where((precio_venta < 1.5 * costo_produccion) & (gasto_publicidad > 2500), 'Éxito', 'Fracaso')
# Si ambas condiciones se cumplen, asigna la etiqueta 'Éxito'. Si no se cumplen, asigna 'Fracaso'.

# Crear DataFrame con los datos
df = pd.DataFrame({
'Costo_de_Produccion': costo_produccion,
'Precio_de_Venta': precio_venta,
'Gasto_en_Publicidad': gasto_publicidad,
'Resultado': etiquetas
})

# Guardar el DataFrame en un archivo CSV
df.to_csv('datos_producto.csv', index=False)



In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Cargar los datos
df = pd.read_csv('datos_producto.csv')

# Preprocesar las etiquetas
df['Resultado'] = np.where(df['Resultado'] == 'Éxito', 1, 0)

# Separar características y etiquetas
X = df[['Costo_de_Produccion', 'Precio_de_Venta', 'Gasto_en_Publicidad']]
y = df['Resultado']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo
rf = RandomForestClassifier(random_state=42)

# Definir los parámetros a ajustar
param_grid = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'min_impurity_decrease': [0.0, 0.01, 0.1]
}

# Configurar Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           scoring='accuracy', cv=5, n_jobs=-1)

# Ajustar el modelo
grid_search.fit(X_train, y_train)

# Resultados
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor precisión en validación cruzada:", grid_search.best_score_)

# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Precisión en conjunto de prueba:", accuracy_score(y_test, y_pred))

